### CRITICAL QUESTION DEFINITION & EVALUATION using SPACY

### 1. schema: Cause to effect CTE

In [86]:
import spacy
import nltk

nltk.download('framenet_v17')

nlp = spacy.load("en_core_web_sm")
from nltk.corpus import framenet as fn

def get_causal_verbs_from_framenet():
    causal_frame_names = [
        "Causation", "Cause_change", "Cause_change_of_position_on_a_scale",
        "Cause_motion", "Cause_to_amalgamate", "Cause_to_start", "Cause_to_make_progress"
    ]

    causal_verbs = set()
    for frame_name in causal_frame_names:
        try:
            frame = fn.frame_by_name(frame_name)
            for lu in frame.lexUnit.values():
                if '.v' in lu['name']:  # Only verbs
                    causal_verbs.add(lu['name'].split('.')[0])
        except Exception as e:
            print(f"Error loading frame '{frame_name}': {e}")
    
    return causal_verbs

causal_verbs = get_causal_verbs_from_framenet()

def detect_cause_to_effect(sentence):
    doc = nlp(sentence)
    explanations = []
    score = 0

    has_condition = any(tok.dep_ == "mark" and tok.text.lower() in {"if", "when"} for tok in doc)
    if has_condition:
        explanations.append("✓ Conditional clause detected (e.g., 'if', 'when')")
        score += 2

    has_advcl = any(tok.dep_ == "advcl" for tok in doc)
    if has_advcl:
        explanations.append("✓ Adverbial clause (likely effect clause) detected")
        score += 2

    has_causal_verb_structure = False

    for tok in doc:
        if tok.lemma_ in causal_verbs and tok.pos_ == "VERB":
            subj = any(child.dep_ == "nsubj" for child in tok.children)
            obj = any(child.dep_ == "dobj" for child in tok.children)
            prep = any(child.dep_ == "prep" for child in tok.children)
            if subj or obj or prep:
                has_causal_verb_structure = True
                explanations.append(
                    f"✓ Verb '{tok.lemma_}' is listed in FrameNet under causal frames and appears with subject/object – "
                    f"possible causality depending on context"
                )
                score += 2
                if subj: score += 1
                if obj: score += 1
                if prep: score += 1
                break

    is_causal = has_condition and has_advcl or has_causal_verb_structure

    if not is_causal:
        causal_phrases = ["result in", "lead to", "cause", "because of", "due to"]
        if any(phrase in sentence.lower() for phrase in causal_phrases):
            explanations.append("✓ Phrase pattern matches known cause-to-effect trigger")
            is_causal = True
            score += 1

    # Clamp max score to 10
    score = min(score, 10)
    return is_causal, explanations, score


def get_causal_verbs_from_framenet():
    # TODO: defined FrameNet frames related to causation -> to check!! Muss man evtl noch eingrenzen
    causal_frame_names = [
        "Causation", "Cause_change", "Cause_change_of_position_on_a_scale",
        "Cause_motion", "Cause_to_amalgamate", "Cause_to_start", "Cause_to_make_progress"
    ]

    causal_verbs = set()

    for frame_name in causal_frame_names:
        frame = fn.frame_by_name(frame_name)
        for lu in frame.lexUnit.values():
            if '.v' in lu['name']:  # Only verbs
                causal_verbs.add(lu['name'].split('.')[0])
    
    return causal_verbs

sentences = [
    "What would happen if taxes increased?",                         # CTE
    "Could rising sea levels result in frequent flooding?",          # CTE
    "Why do doctors recommend exercise?",                             # no
    "Did the rain cause the game to be canceled?",                   #CTE
    "What’s the reason for lowering interest rates?",                # no
    "Is it sunny today or just bright outside?",                     # no
    "What happens if students don’t study?",                         # CTE
    "Why do you prefer chocolate over vanilla?",                     # no
]


for sentence in sentences:
    result, explanation, score = detect_cause_to_effect(sentence)
    if score >= 7:
        label = "Strong CauseToEffect"
    elif score >= 4:
        label = "Weak/Partial CauseToEffect"
    else:
        label = "Not CauseToEffect"
    print(f"\nSentence: {sentence}")
    print(f"CTE: {score}/10 → Label: {label}")
    for e in explanation:
        print(f"   {e}")


[nltk_data] Downloading package framenet_v17 to
[nltk_data]     /Users/lorenaraichle/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!



Sentence: What would happen if taxes increased?
CTE: 7/10 → Label: Strong CauseToEffect
   ✓ Conditional clause detected (e.g., 'if', 'when')
   ✓ Adverbial clause (likely effect clause) detected
   ✓ Verb 'increase' is listed in FrameNet under causal frames and appears with subject/object – possible causality depending on context

Sentence: Could rising sea levels result in frequent flooding?
CTE: 1/10 → Label: Not CauseToEffect
   ✓ Phrase pattern matches known cause-to-effect trigger

Sentence: Why do doctors recommend exercise?
CTE: 0/10 → Label: Not CauseToEffect

Sentence: Did the rain cause the game to be canceled?
CTE: 3/10 → Label: Not CauseToEffect
   ✓ Verb 'cause' is listed in FrameNet under causal frames and appears with subject/object – possible causality depending on context

Sentence: What’s the reason for lowering interest rates?
CTE: 3/10 → Label: Not CauseToEffect
   ✓ Verb 'lower' is listed in FrameNet under causal frames and appears with subject/object – possible 

### SOCRATIC dataset filtering

In [3]:
import pandas as pd
import spacy

# filter relevant context length 
df = pd.read_csv("/Users/lorenaraichle/Developer/NLP-project_v1/clin_IQ/data/processed/SocratiQ/train_chunk_I.csv", names=["category", "context", "question"])
df["context_token_len"] = df["context"].apply(lambda text: len(nlp(text)))
filtered_df = df[df["context_token_len"] >= 25].copy()

print(f"Total rows: {len(df)}")
print(f"Rows after filtering: {len(filtered_df)}")


Total rows: 28195
Rows after filtering: 26804


In [4]:
filtered_df.head()

,category,context,question,context_token_len
1,0.0,alternate_viewpoints_perspectives: A parallel ...,What about nations who have nothing?,126
2,1.0,alternate_viewpoints_perspectives: It would be...,"If not, what about this is cringe exactly?",50
3,2.0,alternate_viewpoints_perspectives: I do not un...,What about public surveillance cameras?,50
4,3.0,alternate_viewpoints_perspectives: There is a ...,How about allowing some students to go straigh...,222
5,4.0,alternate_viewpoints_perspectives: You conside...,What else do you imagine is necessary to be co...,144


- test CauseToEffect performance 

In [5]:

def apply_cte_detection(question):
    result, explanations, score = detect_cause_to_effect(question)
    if score >= 7:
        label = "Strong CauseToEffect"
    elif score >= 4:
        label = "Weak/Partial CauseToEffect"
    else:
        label = "Not CauseToEffect"

    return pd.Series({
        "cte_score": score,
        "cte_label": label,
        "cte_explanations": " | ".join(explanations)  
    })

filtered_df[["cte_score", "cte_label", "cte_explanations"]] = filtered_df["question"].apply(apply_cte_detection)


In [6]:
filtered_df

,category,context,question,context_token_len,cte_score,cte_label,cte_explanations
1,0.0,alternate_viewpoints_perspectives: A parallel ...,What about nations who have nothing?,126,0,Not CauseToEffect,
2,1.0,alternate_viewpoints_perspectives: It would be...,"If not, what about this is cringe exactly?",50,4,Weak/Partial CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
3,2.0,alternate_viewpoints_perspectives: I do not un...,What about public surveillance cameras?,50,0,Not CauseToEffect,
4,3.0,alternate_viewpoints_perspectives: There is a ...,How about allowing some students to go straigh...,222,3,Not CauseToEffect,✓ Verb 'join' is listed in FrameNet under caus...
5,4.0,alternate_viewpoints_perspectives: You conside...,What else do you imagine is necessary to be co...,144,0,Not CauseToEffect,
...,...,...,...,...,...,...,...
28190,28189.0,clarity: Look I am apathetic to what people wa...,Is it really that much effort to call someone ...,205,0,Not CauseToEffect,
28191,28190.0,clarity: I dont think that partly grounding yo...,How is this similar to the issue of individual...,188,0,Not CauseToEffect,
28192,28191.0,clarity: They don’t have cultural impact? It m...,Which part is the conspiracy?,54,0,Not CauseToEffect,
28193,28192.0,clarity: My entire argument is demonstrated by...,Do you think that you are just talking about a...,64,0,Not CauseToEffect,


In [7]:
filtered_df["cte_label"].value_counts()

cte_label
Not CauseToEffect             24159
Weak/Partial CauseToEffect     2504
Strong CauseToEffect            141
Name: count, dtype: int64

In [9]:
strong_cte_df = filtered_df[filtered_df["cte_label"] == "Strong CauseToEffect"].copy()
strong_cte_df

,category,context,question,context_token_len,cte_score,cte_label,cte_explanations
213,212.0,alternate_viewpoints_perspectives: I'd certain...,What about him owning 40 guns made him more da...,126,7,Strong CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
257,256.0,"alternate_viewpoints_perspectives: Yes, I coul...",Do you disagree though that there are strategi...,95,7,Strong CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
504,503.0,"alternate_viewpoints_perspectives: Yea, you co...","If your belief that if someone has a high BMI,...",134,7,Strong CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
837,836.0,alternate_viewpoints_perspectives: The purpose...,How else do people make their decision on who ...,56,8,Strong CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
1002,1001.0,"alternate_viewpoints_perspectives: ""Gender is ...",What else would it mean for a man to live as a...,90,8,Strong CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
...,...,...,...,...,...,...,...
27287,27286.0,clarity: First off I don't have a damn thing a...,How can you mean then that tattoos imply some ...,159,7,Strong CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
27358,27357.0,"clarity: I was looking for your definition, be...",If you have the wrong hormones running through...,171,7,Strong CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
27690,27689.0,clarity: Businesses need employees. No matter ...,So you mean starting a business is hard and mi...,52,7,Strong CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
28010,28009.0,clarity: What? So we're on a 24 hour rule now?...,So it is ok to ask again if circumstances change?,41,7,Strong CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."


In [10]:
strong_cte_df = filtered_df[filtered_df["cte_label"] == "Weak/Partial CauseToEffect"].copy()
strong_cte_df

,category,context,question,context_token_len,cte_score,cte_label,cte_explanations
2,1.0,alternate_viewpoints_perspectives: It would be...,"If not, what about this is cringe exactly?",50,4,Weak/Partial CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
14,13.0,alternate_viewpoints_perspectives: I've been i...,Have you considered that these were things she...,91,5,Weak/Partial CauseToEffect,✓ Adverbial clause (likely effect clause) dete...
57,56.0,alternate_viewpoints_perspectives: From my und...,Do you see any protests in any other cities th...,77,4,Weak/Partial CauseToEffect,✓ Verb 'see' is listed in FrameNet under causa...
86,85.0,alternate_viewpoints_perspectives: If you care...,How else are you suppose to change laws in a m...,81,4,Weak/Partial CauseToEffect,✓ Verb 'change' is listed in FrameNet under ca...
96,95.0,"alternate_viewpoints_perspectives: Right, but ...",Do you know of any other business in the entir...,68,4,Weak/Partial CauseToEffect,✓ Verb 'increase' is listed in FrameNet under ...
...,...,...,...,...,...,...,...
28139,28138.0,clarity: I do not consider it valueless at all...,But you just said that a person who made no fi...,68,4,Weak/Partial CauseToEffect,✓ Verb 'make' is listed in FrameNet under caus...
28142,28141.0,clarity: Saying « my body my choice » just doe...,And would you say that If the sex was consensu...,99,4,Weak/Partial CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
28169,28168.0,"clarity: By ""intellectual property"" I will mea...","If so, how is real property any different from...",64,4,Weak/Partial CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."
28173,28172.0,clarity: I didn't say that honoring him was a ...,"If Stewart and Colbert are pioneers, what the ...",95,6,Weak/Partial CauseToEffect,"✓ Conditional clause detected (e.g., 'if', 'wh..."


- LLM validation of CTE labels only on target rows (with a syntactically indication)

In [11]:
target_rows = filtered_df[
    filtered_df["cte_label"].isin(["Weak/Partial CauseToEffect", "Strong CauseToEffect"])
].copy()


In [12]:
from transformers import pipeline
from huggingface_hub import login
import os
import dotenv
dotenv.load_dotenv()


hf_token = os.getenv("HUGGINGFACE_API_TOKEN")
login(token=hf_token)


def prompt_cte_judgment(context, question, score, label):
    return f"""
You are an expert in argumentation theory.

Context:
{context}

Question:
{question}

A rule-based system has analyzed this question and determined:
- CauseToEffect Score: {score}/10
- Heuristic Label: {label}

The CauseToEffect score reflects structural indicators (e.g., conditional clauses, causal verbs, prepositions).

Does this question genuinely reflect a Cause-to-Effect relationship? For example, does it suggest that one event or condition leads to another?

Use the following labels:
- Confirmed CauseToEffect → Strong evidence of causal reasoning
- Plausible CauseToEffect → Some indicators but less clear
- Not CauseToEffect → No signs of causal structure
- Needs Human Review → Ambiguous or depends heavily on context

Answer:
""".strip()



llm = pipeline(
    "text-generation",
    model="tiiuae/falcon-rw-1b",
    device=-1
)


def llm_evaluate_cte_open(context, question, score, label):  
    prompt = prompt_cte_judgment(context, question, score, label)
    response = llm(prompt, max_new_tokens=50, do_sample=False)
    reply = response[0]["generated_text"].split("Answer:")[-1].strip()
    return reply


/Users/lorenaraichle/Developer/NLP-project_v1/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [13]:
subset_df = target_rows.sample(n=200, random_state=42).copy()
subset_df.shape

(200, 7)

In [14]:
from tqdm import tqdm
tqdm.pandas()
subset_df = target_rows.sample(n=200, random_state=42).copy()
subset_df["llm_cte_label"] = subset_df.progress_apply(
    lambda row: llm_evaluate_cte_open(row["context"], row["question"], row["cte_score"], row["cte_label"]),
    axis=1
)


100%|██████████| 200/200 [22:23<00:00,  6.72s/it]


In [16]:

subset_df.to_excel("train_chunk_1_CTE.xlsx", index=False)

subset_df

### 2. schema: Expert opinion

In [29]:
import spacy
import nltk
from nltk.corpus import framenet as fn

nltk.download('framenet_v17')
nlp = spacy.load("en_core_web_sm")

# ----------- Generalized FrameNet Loader -----------
def get_lexical_units_from_frames(frames):
    terms = set()
    for frame_name in frames:
        try:
            frame = fn.frame_by_name(frame_name)
            for lu in frame.lexUnit.values():
                if '.v' in lu['name']:
                    terms.add(lu['name'].split('.')[0])
        except Exception as e:
            print(f"Warning: Could not load frame '{frame_name}': {e}")
    return terms

# ----------- Extract FrameNet-based Verb Sets -----------

expert_frames = [
    "Expertise", "Judgment_communication", "Opinion",
    "Authority", "Statement", "Certainty"
]
quote_frames = ["Statement", "Judgment_communication"]
clarity_frames = ["Reasoning"]
evidence_frames = ["Evidence", "Certainty", "Causation"]


expert_verbs = get_lexical_units_from_frames(expert_frames)
quote_verbs = get_lexical_units_from_frames(quote_frames)
clarity_terms = get_lexical_units_from_frames(clarity_frames)
evidence_terms = get_lexical_units_from_frames(evidence_frames)

# ----------- Detection Function -----------

def detect_expert_opinion(question):
    doc = nlp(question)
    score = 0
    explanations = []

    expert_titles = {"expert", "researcher", "scientist", "doctor", "analyst", "professor", "Dr."}

    for ent in doc.ents:
        if ent.label_ in {"PERSON", "ORG"}:
            if any(title in ent.text.lower() for title in expert_titles):
                explanations.append(f"✓ Expert entity detected: '{ent.text}'")
                score += 2
                break

    if any(tok.lemma_ in expert_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Detected expert-related verb from FrameNet")
        score += 2

    if any(tok.lemma_ in quote_verbs for tok in doc):
        explanations.append("✓ Quotation or claim verb found")
        score += 1

    if any(tok.lemma_ in clarity_terms for tok in doc):
        explanations.append("✓ Clarity/definition markers found")
        score += 1

    if any(tok.lemma_ in evidence_terms for tok in doc):
        explanations.append("✓ Evidence or support-related terms found")
        score += 2

    if score >= 6:
        label = "Strong Expert Opinion"
    elif score >= 3:
        label = "Weak/Partial Expert Opinion"
    else:
        label = "Not Expert Opinion"

    return label, score, explanations

# ----------- Example Usage -----------

questions = [
    "Did Professor Lee actually say that the results were inconclusive?",
    "Is this study consistent with what other experts have found?",
    "Why do cats sleep so much?",
    "Are there any recent studies supporting this?",
    "What did the government state about inflation?",
    "Is the evidence consistent with past research?",
    "Can you define the technical term 'quantum entanglement'?",
    "Who won the World Cup in 2018?"
]

for q in questions:
    label, score, explanation = detect_expert_opinion(q)
    print(f"\nQuestion: {q}")
    print(f"Label: {label} | Score: {score}/10")
    for e in explanation:
        print(f"   {e}")


[nltk_data] Downloading package framenet_v17 to
[nltk_data]     /Users/lorenaraichle/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!



Question: Did Professor Lee actually say that the results were inconclusive?
Label: Weak/Partial Expert Opinion | Score: 3/10
   ✓ Detected expert-related verb from FrameNet
   ✓ Quotation or claim verb found

Question: Is this study consistent with what other experts have found?
Label: Not Expert Opinion | Score: 0/10

Question: Why do cats sleep so much?
Label: Not Expert Opinion | Score: 0/10

Question: Are there any recent studies supporting this?
Label: Not Expert Opinion | Score: 2/10
   ✓ Evidence or support-related terms found

Question: What did the government state about inflation?
Label: Not Expert Opinion | Score: 1/10
   ✓ Quotation or claim verb found

Question: Is the evidence consistent with past research?
Label: Not Expert Opinion | Score: 2/10
   ✓ Evidence or support-related terms found

Question: Can you define the technical term 'quantum entanglement'?
Label: Not Expert Opinion | Score: 0/10

Question: Who won the World Cup in 2018?
Label: Not Expert Opinion | Sco

In [30]:

filtered_df[["expert_label", "expert_score", "expert_explanations"]] = filtered_df["question"].apply(
    lambda q: pd.Series(detect_expert_opinion(q))
)


In [31]:
filtered_df["expert_label"].value_counts()

expert_label
Not Expert Opinion             23343
Weak/Partial Expert Opinion     3442
Strong Expert Opinion             19
Name: count, dtype: int64

In [32]:
strong_expert_df = filtered_df[filtered_df["expert_label"] == "Strong Expert Opinion"].copy()
strong_expert_df

,category,context,question,context_token_len,cte_score,cte_label,cte_explanations,expert_label,expert_score,expert_explanations
296,295.0,alternate_viewpoints_perspectives: This isn't ...,What about someone who is explaining their rea...,203,2,Not CauseToEffect,✓ Adverbial clause (likely effect clause) dete...,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."
936,935.0,alternate_viewpoints_perspectives: The problem...,Would you not rather say that voter fraud has ...,36,0,Not CauseToEffect,,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."
941,940.0,alternate_viewpoints_perspectives: Here's my a...,Is it not possible to prove unobservable thing...,34,0,Not CauseToEffect,,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."
1560,1559.0,"alternate_viewpoints_perspectives: Again, I li...",How about any video evidence that shows the bo...,73,0,Not CauseToEffect,,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."
3255,3254.0,alternate_viewpoints_perspectives: Except that...,How about empathy until proven otherwise for t...,57,2,Not CauseToEffect,✓ Adverbial clause (likely effect clause) dete...,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."
4255,4254.0,alternate_viewpoints_perspectives: When a whit...,Have you ever considered that the reason peopl...,120,4,Weak/Partial CauseToEffect,✓ Verb 'make' is listed in FrameNet under caus...,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."
6060,6059.0,assumptions: They'd hold that they're jumping ...,Are you saying that we should assume a white o...,63,2,Not CauseToEffect,✓ Adverbial clause (likely effect clause) dete...,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."
9783,9782.0,"clarity: Well, you could say that a self-contr...",Are you trying to say that reason being object...,82,0,Not CauseToEffect,,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."
10774,10773.0,clarity: The lack of evidence of a god is prob...,Is it not the burden of the accuser to prove t...,62,0,Not CauseToEffect,,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."
11378,11377.0,clarity: I’ll list several examples. A- Black ...,Are you referring to the self described white ...,46,0,Not CauseToEffect,,Strong Expert Opinion,6,"[✓ Detected expert-related verb from FrameNet,..."


# 3. schema: analogy

In [79]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

# Get synsets for analogy-related terms
analogy_synsets = [wn.synset('similar.a.01'), wn.synset('analogy.n.01'), wn.synset('compare.v.01')]

def is_semantically_analogical(token):
    token_synsets = wn.synsets(token.lemma_)
    for s in token_synsets:
        for analogy_syn in analogy_synsets:
            if s.path_similarity(analogy_syn) and s.path_similarity(analogy_syn) > 0.3:
                return True
    return False

def get_lexical_units_from_frames(frames):
    terms = set()
    for frame_name in frames:
        try:
            frame = fn.frame_by_name(frame_name)
            for lu in frame.lexUnit.values():
                if '.v' in lu['name']:
                    terms.add(lu['name'].split('.')[0])
        except Exception as e:
            print(f"Warning: Could not load frame '{frame_name}': {e}")
    return terms

# FrameNet sets
comparison_frames = ["Similarity"]
contrast_frames = ["Categorization"]
evidence_frames = ["Evidence", "Judgment_communication"]

comparison_verbs = get_lexical_units_from_frames(comparison_frames)
contrast_verbs = get_lexical_units_from_frames(contrast_frames)
evidence_verbs = get_lexical_units_from_frames(evidence_frames)

# Analogy detection function
def detect_analogy_question(question):
    doc = nlp(question)
    score = 0
    explanations = []
    noun_chunks = list(doc.noun_chunks)

    # Comparison markers
    if any(tok.lemma_ in comparison_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Comparison verb detected from FrameNet")
        score += 2
    
    #pairwise comprison
    entity_tokens = [tok for tok in doc if tok.pos_ in {"PROPN", "NOUN"}]
    if len(set([tok.lemma_ for tok in entity_tokens])) >= 2:
        score += 1
        explanations.append("✓ Contains at least two distinct concepts/entities")

    # Contrast markers
    if any(tok.lemma_ in contrast_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Contrast or difference verb detected from FrameNet")
        score += 1

    # Evidence markers
    if any(tok.lemma_ in evidence_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Evidence or justification verb found")
        score += 1
        
    if any(tok.tag_ in {"MD"} for tok in doc):  # Modal verbs like would, could
        score += 1

    if len(noun_chunks) >= 2 and any(tok.lemma_ in {"similar", "like", "as"} for tok in doc):
        explanations.append("✓ Two concepts compared with similarity cue (e.g., 'similar', 'like')")
        score += 2


    # Conditional marker (e.g., 'if')
    if any(tok.text.lower() == "if" for tok in doc):
        explanations.append("✓ Conditional structure suggesting hypothetical reasoning")
        score += 1
        
    if any(is_semantically_analogical(tok) for tok in doc if tok.pos_ in {"ADJ", "NOUN", "VERB"}):
        explanations.append("✓ Semantic similarity to analogy-related terms detected via WordNet")
        score += 2
        
    if any(tok.dep_ in {"prep", "relcl"} and tok.lemma_ in {"compare", "similar"} for tok in doc):
        score += 1
        explanations.append("✓ Syntactic cue of analogy (e.g., 'compared with', 'similar to')")



    # Label assignment
    if score >= 8:
        label = "Strong Analogy Question"
    elif score >= 5:
        label = "Weak/Partial Analogy Question"
    else:
        label = "Not Analogy Question"

    return label, score, explanations


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lorenaraichle/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [80]:
questions = [
    "Are cats and dogs similar in how they form social bonds?",
    "Is democracy in the U.S. similar to that in ancient Greece?",
    "If Finland succeeded with this policy, would it work in Germany?",
    "What did Plato say about justice?",
    "Are there differences between ancient Rome and the modern EU?",
    "Who won the match yesterday?"
]

for q in questions:
    label, score, explanation = detect_analogy_question(q)
    print(f"\nQuestion: {q}")
    print(f"Label: {label} | Score: {score}/10")
    for e in explanation:
        print(f"   {e}")



Question: Are cats and dogs similar in how they form social bonds?
Label: Weak/Partial Analogy Question | Score: 5/10
   ✓ Contains at least two distinct concepts/entities
   ✓ Two concepts compared with similarity cue (e.g., 'similar', 'like')
   ✓ Semantic similarity to analogy-related terms detected via WordNet

Question: Is democracy in the U.S. similar to that in ancient Greece?
Label: Weak/Partial Analogy Question | Score: 5/10
   ✓ Contains at least two distinct concepts/entities
   ✓ Two concepts compared with similarity cue (e.g., 'similar', 'like')
   ✓ Semantic similarity to analogy-related terms detected via WordNet

Question: If Finland succeeded with this policy, would it work in Germany?
Label: Weak/Partial Analogy Question | Score: 5/10
   ✓ Contains at least two distinct concepts/entities
   ✓ Conditional structure suggesting hypothetical reasoning
   ✓ Semantic similarity to analogy-related terms detected via WordNet

Question: What did Plato say about justice?
Label

In [85]:

df = pd.read_csv("/Users/lorenaraichle/Developer/NLP-project_v1/clin_IQ/data/processed/SocratiQ/train_chunk_I.csv", names=["category", "context", "question"])
df["context_token_len"] = df["context"].apply(lambda text: len(nlp(text)))
df_filtered = df[df["context_token_len"] >= 25].copy()

print(f"Total rows: {len(df)}")

Total rows: 28195


In [103]:
df_filtered[["analogy_label", "analogy_score", "analogy_explanations"]] = df_filtered["question"].apply(
    lambda q: pd.Series(detect_analogy_question(q))
)

In [105]:
df_filtered["analogy_label"].value_counts()


analogy_label
Not Analogy Question             25019
Weak/Partial Analogy Question     1773
Strong Analogy Question             12
Name: count, dtype: int64

In [106]:
strong_analogy_df = df_filtered[df_filtered["analogy_label"] == "Strong Analogy Question"].copy()
strong_analogy_df

,category,context,question,context_token_len,fear_label,fear_score,fear_explanations,analogy_label,analogy_score,analogy_explanations
1002,1001.0,"alternate_viewpoints_perspectives: ""Gender is ...",What else would it mean for a man to live as a...,90,Strong Fear Appeal,6,"[✓ Modal verb detected (e.g., 'might', 'would'...",Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...
1064,1063.0,alternate_viewpoints_perspectives: Since priva...,If private property only exists as a legal con...,170,Not Fear Appeal,3,"[✓ Conditional clause found (e.g., 'if', 'unle...",Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...
5431,5430.0,alternate_viewpoints_perspectives: I believe t...,Have you ever considered these people simply a...,138,Weak/Partial Fear Appeal,5,[✓ Causal/preventive verb from FrameNet detect...,Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...
6109,6108.0,assumptions: I can demand more money then. I f...,And would you not happily assume a share of th...,48,Strong Fear Appeal,6,"[✓ Modal verb detected (e.g., 'might', 'would'...",Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...
6263,6262.0,"assumptions: ""there was no fraud, but even if ...",If the proof is that like 60 votes should have...,84,Weak/Partial Fear Appeal,4,"[✓ Modal verb detected (e.g., 'might', 'would'...",Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...
8015,8014.0,"clarity: For starter, I do think the two men a...",Or do you mean that the issue of implicit bias...,92,Weak/Partial Fear Appeal,5,"[✓ Modal verb detected (e.g., 'might', 'would'...",Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...
9753,9752.0,clarity: The constitution has been amended 27 ...,Which is basically what citizens united w how ...,126,Weak/Partial Fear Appeal,4,"[✓ Modal verb detected (e.g., 'might', 'would'...",Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...
10329,10328.0,clarity: No. It is not their genitalia that ma...,If you would identify people who express mascu...,71,Strong Fear Appeal,6,"[✓ Modal verb detected (e.g., 'might', 'would'...",Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...
22277,22276.0,clarity: Oh my. Okay. I get where you’re going...,So all we have to do is define a slave as less...,40,Not Fear Appeal,3,"[✓ Conditional clause found (e.g., 'if', 'unle...",Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...
24856,24855.0,clarity: In the 2000s and the 10s music hasn't...,Would you count it as new development if one o...,45,Strong Fear Appeal,6,"[✓ Modal verb detected (e.g., 'might', 'would'...",Strong Analogy Question,8,[✓ Contains at least two distinct concepts/ent...


# 4. Bias 


In [91]:
import spacy
import nltk
from nltk.corpus import framenet as fn

# Setup
nltk.download('framenet_v17')
nlp = spacy.load("en_core_web_sm")

from nltk.corpus import wordnet as wn

def is_fear_related(token):
    syns = wn.synsets(token.lemma_)
    for s in syns:
        if any(s.path_similarity(wn.synset('danger.n.01')) or
               s.path_similarity(wn.synset('fear.n.01')) or
               s.path_similarity(wn.synset('threat.n.01')) for s in syns):
            return True
    return False


# ---- FrameNet Utility ----
def get_lexical_units_from_frames(frames):
    terms = set()
    for frame_name in frames:
        try:
            frame = fn.frame_by_name(frame_name)
            for lu in frame.lexUnit.values():
                if '.v' in lu['name']:
                    terms.add(lu['name'].split('.')[0])
        except:
            continue
    return terms

# ---- Relevant Lexical Resources ----
causal_frames = ["Causation", "Cause_to_start", "Preventing", "Risk", "Threaten", "Danger"]
causal_verbs = get_lexical_units_from_frames(causal_frames)

fear_keywords = {"danger", "threat", "risky", "harm", "catastrophe", "crisis", "ruin", "fear", "worse", "bad", "fatal"}
preventive_keywords = {"prevent", "avoid", "stop", "ban", "rescue", "save", "protect"}
modal_keywords = {"might", "could", "would", "may", "should"}

# ---- Detection Function ----
def detect_fear_appeal_question(question):
    doc = nlp(question)
    score = 0
    explanations = []

    # Modal structure
    if any(tok.lemma_ in modal_keywords for tok in doc if tok.tag_ == "MD"):
        explanations.append("✓ Modal verb detected (e.g., 'might', 'would') suggesting hypothetical risk")
        score += 1

    # Threat/danger terms
    if any(tok.lemma_.lower() in fear_keywords for tok in doc):
        explanations.append("✓ Fear-related keyword detected (e.g., 'threat', 'danger')")
        score += 2

    # Prevention-related words
    if any(tok.lemma_.lower() in preventive_keywords for tok in doc):
        explanations.append("✓ Preventive action verb detected (e.g., 'prevent', 'stop')")
        score += 2

    # Causal verbs from FrameNet
    if any(tok.lemma_ in causal_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Causal/preventive verb from FrameNet detected")
        score += 2

    # Hypothetical or conditional reasoning
    if any(tok.text.lower() in {"if", "unless"} for tok in doc):
        explanations.append("✓ Conditional clause found (e.g., 'if', 'unless')")
        score += 1
    
    if any(is_fear_related(tok) for tok in doc if tok.pos_ in {"NOUN", "VERB", "ADJ"}):
        explanations.append("✓ Semantic fear-related concept detected via WordNet")
        score += 2
        
    # Final label
    if score >= 6:
        label = "Strong Fear Appeal"
        
    elif score >= 4:
        label = "Weak/Partial Fear Appeal"
    else:
        label = "Not Fear Appeal"

    return label, score, explanations


[nltk_data] Downloading package framenet_v17 to
[nltk_data]     /Users/lorenaraichle/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!


In [92]:
questions = [
    "If we don't regulate AI, it might take over critical infrastructure.",
    "Is banning TikTok the only way to prevent surveillance?",
    "Could this policy save us from a financial disaster?",
    "Why is inflation bad for the middle class?",
    "Should we stop immigration to prevent job loss?",
    "Who benefits from this healthcare policy?",
    "What happens if we ignore climate change?"
]

for q in questions:
    label, score, explanation = detect_fear_appeal_question(q)
    print(f"\nQuestion: {q}")
    print(f"Label: {label} | Score: {score}/10")
    for e in explanation:
        print(f"   {e}")



Question: If we don't regulate AI, it might take over critical infrastructure.
Label: Weak/Partial Fear Appeal | Score: 4/10
   ✓ Modal verb detected (e.g., 'might', 'would') suggesting hypothetical risk
   ✓ Conditional clause found (e.g., 'if', 'unless')
   ✓ Semantic fear-related concept detected via WordNet

Question: Is banning TikTok the only way to prevent surveillance?
Label: Weak/Partial Fear Appeal | Score: 4/10
   ✓ Preventive action verb detected (e.g., 'prevent', 'stop')
   ✓ Semantic fear-related concept detected via WordNet

Question: Could this policy save us from a financial disaster?
Label: Weak/Partial Fear Appeal | Score: 5/10
   ✓ Modal verb detected (e.g., 'might', 'would') suggesting hypothetical risk
   ✓ Preventive action verb detected (e.g., 'prevent', 'stop')
   ✓ Semantic fear-related concept detected via WordNet

Question: Why is inflation bad for the middle class?
Label: Weak/Partial Fear Appeal | Score: 4/10
   ✓ Fear-related keyword detected (e.g., 'thr

In [101]:
df_filtered[["fear_label", "fear_score", "fear_explanations"]] = df_filtered["question"].apply(
    lambda q: pd.Series(detect_fear_appeal_question(q))
)


In [102]:
df_filtered.value_counts()

TypeError: unhashable type: 'list'

In [98]:
strong_expert_df = df_filtered[df_filtered["fear_label"] == "Strong Fear Appeal"].copy()
strong_expert_df

,category,context,question,context_token_len,fear_label,fear_score,fear_explanations
114,113.0,alternate_viewpoints_perspectives: This post i...,What about the climate people makes them worse?,58,Strong Fear Appeal,6,"[✓ Fear-related keyword detected (e.g., 'threa..."
563,562.0,alternate_viewpoints_perspectives: The rule th...,Is it not the case that you could have avoided...,43,Strong Fear Appeal,6,"[✓ Modal verb detected (e.g., 'might', 'would'..."
744,743.0,alternate_viewpoints_perspectives: The more I ...,Have you considered that the best way to prote...,125,Strong Fear Appeal,6,"[✓ Preventive action verb detected (e.g., 'pre..."
1002,1001.0,"alternate_viewpoints_perspectives: ""Gender is ...",What else would it mean for a man to live as a...,90,Strong Fear Appeal,6,"[✓ Modal verb detected (e.g., 'might', 'would'..."
1428,1427.0,alternate_viewpoints_perspectives: Recently in...,"If you ban that, what else would you base laws...",36,Strong Fear Appeal,6,"[✓ Modal verb detected (e.g., 'might', 'would'..."
...,...,...,...,...,...,...,...
27690,27689.0,clarity: Businesses need employees. No matter ...,So you mean starting a business is hard and mi...,52,Strong Fear Appeal,6,"[✓ Modal verb detected (e.g., 'might', 'would'..."
27703,27702.0,"clarity: For years, the same argument has been...",What makes working conditions bad in your view?,95,Strong Fear Appeal,6,"[✓ Fear-related keyword detected (e.g., 'threa..."
27828,27827.0,clarity: What about research suggesting that p...,Does that mean poor people are to be thought o...,156,Strong Fear Appeal,6,"[✓ Fear-related keyword detected (e.g., 'threa..."
27976,27975.0,clarity: Since her biases are inherently reaso...,Is it really the case that those black people ...,243,Strong Fear Appeal,6,"[✓ Fear-related keyword detected (e.g., 'threa..."


In [99]:
df_filtered["fear_label"].value_counts()

fear_label
Not Fear Appeal             22973
Weak/Partial Fear Appeal     3705
Strong Fear Appeal            126
Name: count, dtype: int64